In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('updated_coswara.csv')

In [2]:
df

,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc{1},mfcc{2},mfcc{3},mfcc{4},...,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,aqpq5Shimmer,apq11Shimmer,ddaShimmer,label
0,0.039102,0.378903,786.823461,966.699650,1387.984940,0.043870,-412.08945,126.752335,31.558170,18.483738,...,0.018672,0.021894,0.056015,0.212818,1.734878,0.097995,0.147546,0.206467,0.293984,1
1,0.051812,0.436672,2219.820298,1874.652272,4134.754998,0.209401,-398.93295,50.929253,-17.480385,4.325164,...,0.030713,0.037553,0.092139,0.283908,2.113383,0.137225,0.241707,0.116884,0.411675,0
2,0.010413,0.238371,2002.598308,2058.223470,4324.443295,0.146994,-599.82200,40.048190,6.373952,13.369130,...,0.018672,0.022602,0.056016,0.134015,1.358899,0.061172,0.091838,0.164672,0.183516,1
3,0.078437,0.242433,569.328347,506.956042,979.591497,0.035271,-423.74878,77.112580,-11.768955,6.080464,...,0.050445,0.070875,0.151334,0.181641,1.673087,0.101366,0.093390,0.116884,0.304098,0
4,0.072712,0.327475,1344.446613,1062.582139,2431.292693,0.089481,-327.21740,160.910540,-60.493977,-20.299130,...,0.001202,0.001537,0.003605,0.085836,0.803849,0.035992,0.054323,0.093975,0.107976,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24123,0.056854,0.467262,2845.574993,2044.704050,5269.369989,0.235494,-388.71793,32.990032,-8.879510,20.782476,...,0.023928,0.029485,0.071784,0.160208,1.659819,0.064901,0.137543,0.331425,0.194703,0
24124,0.050149,0.267105,1199.737564,1636.413852,2392.060470,0.059086,-433.10117,58.741127,37.350792,51.354850,...,0.002124,0.002396,0.006373,0.042722,0.419731,0.018084,0.027000,0.041864,0.054251,0
24125,0.000454,0.525523,4996.270042,2579.510698,8270.021928,0.418827,-745.49110,-27.318123,-5.540486,16.303144,...,0.016868,0.019114,0.050605,0.137923,1.255627,0.061327,0.081247,0.116884,0.183981,0
24126,0.031053,0.167581,1890.217497,2748.644250,5175.546000,0.033493,-499.73740,32.764600,46.792103,40.457275,...,0.001794,0.001854,0.005381,0.016609,0.134111,0.007814,0.009204,0.011837,0.023441,1


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, Activation, Multiply, Permute, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [5]:
# Reshape the features to fit the CNN-LSTM model
X = X.reshape(X.shape[0], X.shape[1], 1)

# Define the input shape
input_shape = (X.shape[1], X.shape[2])


In [6]:
# Define the number of filters and kernel size for the convolutional layers
num_filters = 64
kernel_size = 3

# Define the number of units for the LSTM layer
num_units = 128

# Define the dropout rate
dropout_rate = 0.5

In [7]:
def attention_3d_block(inputs):
    # Compute the score by multiplying the input with a trainable weight vector
    attention_weights = Dense(1, activation='tanh')(inputs)
    # Reshape the score to a vector
    attention_weights = Flatten()(attention_weights)
    # Apply a softmax activation function to the score vector
    attention_weights = Activation('softmax')(attention_weights)
    # Reshape the score vector to a tensor
    attention_weights = Reshape((-1, 1))(attention_weights)
    # Multiply the input with the attention weights
    multiplied = Multiply()([inputs, attention_weights])
    # Compute the output of the attention mechanism
    output = K.sum(multiplied, axis=1)
    return output

In [8]:
# Define the model
input_layer = Input(shape=input_shape)
conv1 = Conv1D(num_filters, kernel_size, activation='relu')(input_layer)
maxpool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool1)
maxpool2 = MaxPooling1D(pool_size=2)(conv2)
lstm_layer = LSTM(num_units, return_sequences=True)(maxpool2)
attention_layer = attention_3d_block(lstm_layer)
dropout_layer = Dropout(dropout_rate)(attention_layer)
flatten_layer = Flatten()(dropout_layer)
output_layer = Dense(1, activation='sigmoid')(flatten_layer)
model = Model(inputs=input_layer, outputs=output_layer)

In [9]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [10]:
# Train the model
epochs = 20
batch_size = 32
history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

loss, accuracy = model.evaluate(X, y)
print(f"Test accuracy: {accuracy}")

Epoch 1/20
604/604 [==============================] - 50s 50ms/step - loss: 0.6740 - accuracy: 0.5792 - val_loss: 0.6329 - val_accuracy: 0.6577
Epoch 2/20
604/604 [==============================] - 21s 35ms/step - loss: 0.5851 - accuracy: 0.6883 - val_loss: 0.5698 - val_accuracy: 0.7186
Epoch 3/20
604/604 [==============================] - 26s 44ms/step - loss: 0.5214 - accuracy: 0.7383 - val_loss: 0.4904 - val_accuracy: 0.7576
Epoch 4/20
604/604 [==============================] - 27s 45ms/step - loss: 0.5022 - accuracy: 0.7499 - val_loss: 0.5103 - val_accuracy: 0.7464
Epoch 5/20
604/604 [==============================] - 27s 44ms/step - loss: 0.4886 - accuracy: 0.7613 - val_loss: 0.5813 - val_accuracy: 0.6846
Epoch 6/20
604/604 [==============================] - 27s 45ms/step - loss: 0.4819 - accuracy: 0.7620 - val_loss: 0.4844 - val_accuracy: 0.7698
Epoch 7/20
604/604 [==============================] - 28s 47ms/step - loss: 0.4788 - accuracy: 0.7652 - val_loss: 0.5045 - val_accuracy:

In [17]:
# Define the model
input_layer = Input(shape=input_shape)
conv1 = Conv1D(num_filters, kernel_size, activation='relu')(input_layer)
maxpool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool1)
maxpool2 = MaxPooling1D(pool_size=2)(conv2)
conv3 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool2)
maxpool3 = MaxPooling1D(pool_size=2)(conv3)
lstm_layer = LSTM(num_units, return_sequences=True)(maxpool3)
lstm_layer2 = LSTM(num_units, return_sequences=True)(lstm_layer)
attention_layer = attention_3d_block(lstm_layer2)
dropout_layer = Dropout(dropout_rate)(attention_layer)
flatten_layer = Flatten()(dropout_layer)
output_layer = Dense(1, activation='sigmoid')(flatten_layer)
model = Model(inputs=input_layer, outputs=output_layer)

In [18]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [19]:
epochs = 20
batch_size = 32
history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

loss, accuracy = model.evaluate(X, y)
print(f"Test accuracy: {accuracy}")

Epoch 1/20
604/604 [==============================] - 72s 54ms/step - loss: 0.6452 - accuracy: 0.6155 - val_loss: 0.6099 - val_accuracy: 0.6732
Epoch 2/20
604/604 [==============================] - 31s 51ms/step - loss: 0.5139 - accuracy: 0.7440 - val_loss: 0.5401 - val_accuracy: 0.7350
Epoch 3/20
604/604 [==============================] - 31s 51ms/step - loss: 0.4832 - accuracy: 0.7625 - val_loss: 0.4692 - val_accuracy: 0.7694
Epoch 4/20
604/604 [==============================] - 28s 47ms/step - loss: 0.4711 - accuracy: 0.7719 - val_loss: 0.4648 - val_accuracy: 0.7795
Epoch 5/20
604/604 [==============================] - 29s 48ms/step - loss: 0.4616 - accuracy: 0.7779 - val_loss: 0.4989 - val_accuracy: 0.7445
Epoch 6/20
604/604 [==============================] - 28s 46ms/step - loss: 0.4589 - accuracy: 0.7776 - val_loss: 0.4563 - val_accuracy: 0.7808
Epoch 7/20
604/604 [==============================] - 34s 56ms/step - loss: 0.4472 - accuracy: 0.7832 - val_loss: 0.4559 - val_accuracy:

In [20]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X, y, verbose=0)

# Predict the labels of the test set
y_pred = model.predict(X)

# Convert the predicted probabilities to labels
y_pred_labels = (y_pred > 0.5).astype(int)

# Calculate the performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

accuracy = accuracy_score(y, y_pred_labels)
precision = precision_score(y, y_pred_labels)
recall = recall_score(y, y_pred_labels)
f1_score = f1_score(y, y_pred_labels)
roc_auc = roc_auc_score(y, y_pred)
tn, fp, fn, tp = confusion_matrix(y, y_pred_labels).ravel()
specificity = tn / (tn + fp)


754/754 [==============================] - 17s 18ms/step


In [21]:
print('Accuracy: ', accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('f1_score',f1_score)
print('AUC: ', roc_auc)
print('Specificity: ', specificity)


Accuracy:  0.821949602122016
Precision:  0.8967325042672519
Recall:  0.6807663828211773
f1_score 0.7739661159633799
AUC:  0.894014925590453
Specificity:  0.9364305013509456


In [27]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Embedding, Bidirectional, Conv1D, MaxPooling1D, Flatten, Activation, multiply, Lambda
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix



# Define the metrics to be calculated
metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1_score': [], 'roc_auc': []}

# Perform 3-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
for fold, (train, test) in enumerate(kfold.split(X, y)):
    print('Fold:', fold+1)
    
    # Define the model for this fold
    conv1 = Conv1D(num_filters, kernel_size, activation='relu')(input_layer)
    maxpool1 = MaxPooling1D(pool_size=2)(conv1)
    conv2 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool1)
    maxpool2 = MaxPooling1D(pool_size=2)(conv2)
    conv3 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool2)
    maxpool3 = MaxPooling1D(pool_size=2)(conv3)
    lstm_layer = LSTM(num_units, return_sequences=True)(maxpool3)
    lstm_layer2 = LSTM(num_units, return_sequences=True)(lstm_layer)
    attention_layer = attention_3d_block(lstm_layer2)
    dropout_layer = Dropout(dropout_rate)(attention_layer)
    flatten_layer = Flatten()(dropout_layer)
    output_layer = Dense(1, activation='sigmoid')(flatten_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train the model on the training data for this fold
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    
    # Make predictions on the test data for this fold
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).flatten()
    
    # Calculate the metrics for this fold and append to the list of metrics
    metrics['accuracy'].append(accuracy_score(y_test, y_pred))
    metrics['precision'].append(precision_score(y_test, y_pred))
    metrics['recall'].append(recall_score(y_test, y_pred))
    metrics['f1_score'].append(f1_score(y_test, y_pred))
    metrics['roc_auc'].append(roc_auc_score(y_test, y_pred))
    
    # Print the confusion matrix for this fold
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))
    print()

# Calculate and print the average metrics over all folds
for metric in metrics:
    print(metric, np.mean(metrics[metric]))


Fold: 1
252/252 [==============================] - 6s 17ms/step
Confusion Matrix:
[[3400 1041]
 [ 817 2785]]

Fold: 2
252/252 [==============================] - 11s 22ms/step
Confusion Matrix:
[[3353 1089]
 [ 842 2759]]

Fold: 3
252/252 [==============================] - 11s 23ms/step
Confusion Matrix:
[[4202  239]
 [1463 2138]]

accuracy 0.7724227428034399
precision 0.7814544016358881
recall 0.7110271978556145
f1_score 0.7353084473123893
roc_auc 0.7666163970903049


In [28]:
print(metrics)

{'accuracy': [0.7689916697749596, 0.7599154544324257, 0.7883611042029346], 'precision': [0.7279142707788814, 0.716995841995842, 0.8994530921329407], 'recall': [0.7731815657967795, 0.766176062204943, 0.5937239655651207], 'f1_score': [0.7498653742595585, 0.7407705732313064, 0.7152893944463031], 'roc_auc': [0.7693874503156382, 0.7605081121470461, 0.7699536288082303]}
